In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/aditya/qaisw-v2.15.1.230926150623_62883"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw"#rawfiles
os.environ['FOLDER_WITH_ARTIFACTS']="YoloX_updated"
os.environ['DLC32']="yolox_FP32_2_15_1.dlc"
os.environ['DLC8']="yolox_a8w8_2_15_1.dlc"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['ONDEVICE_FOLDER']="YoloX_updated"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="728b7a92" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
import cv2
import glob
import os
import numpy as np
from PIL import Image
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

### ONNX Model

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.onnx

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc --input_network yolox_x.onnx --output_path yolox_FP32_2_15_1.dlc

### DLC Conversion

- Please create the dlc models(FP32) from the generating_model.ipynb file
- After Creating the Raw file here we'll generate the INT8 Model

# Preprocessing

### Getting the dataset


In [ ]:
# User needs to download the dataset of their choice here. 

In [ ]:
# dataset is huge to run on device , taking only 15 dataset
files = os.listdir('val2017')
for file in files[15:]:
    os.remove("val2017/"+file)

In [ ]:
def preproc(img, input_size, swap=(2, 0, 1)):
    if len(img.shape) == 3:
        padded_img = np.ones((input_size[0], input_size[1], 3), dtype=np.uint8) * 114
    else:
        padded_img = np.ones(input_size, dtype=np.uint8) * 114

    r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
    resized_img = cv2.resize(
        img,
        (int(img.shape[1] * r), int(img.shape[0] * r)),
        interpolation=cv2.INTER_LINEAR,
    ).astype(np.uint8)
    padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img

    padded_img = padded_img.transpose(swap)
    padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)
    return padded_img, r

In [ ]:
import cv2
import numpy as np
import os
dict={}
input_shape = tuple(map(int,[640,640]))  
def detect(imgfile,i):
    origimg = cv2.imread(imgfile)
    img, ratio = preproc(origimg, input_shape)
    index=imgfile.split("/")[1].split(".")[0]
    dict[index]=ratio
    img=np.transpose(img,(1,2,0))
    img.tofile("raw/"+filenames[i].split(".")[0]+".raw")
    
filenames = os.listdir("val2017")
for i in range(0,len(filenames)):
    if "jpg" in filenames[i].lower():
        detect("val2017/"+filenames[i],i)

### Creating the list file

In [ ]:
%%bash
find ./raw -name *.raw > list.txt

#### Creating the INT 8 Model After preparing the raw files

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-quantize --input_dlc yolox_FP32_2_15_1.dlc --input_list list.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc yolox_a8w8_2_15_1.dlc --enable_htp --htp_socs sm8550

#### Creating Bin and Lib Folder on Device 

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

### Pushing all Lib and Bin files onto Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v75/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

### Pushing Artifacts on to Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push raw /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

### Inferencing 8 bit DLC on DSP Runtime
* Give name of DLC in OUTPUT_DLC_QUANTIZED8 and ondevice folder

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=yolox_a8w8_2_15_1.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="YoloX_updated"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list list.txt --output_dir $OUTPUT_FOLDER --use_dsp

### Inferencing 32b DLC on CPU Runtime
Give name of DLC in OUTPUT_DLC_32 and ondevice folder

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=yolox_FP32_2_15_1.dlc
export ONDEVICE_FOLDER="YoloX_updated"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list list.txt --output_dir $OUTPUT_FOLDER

### Pulling Output folder generated on different precision and cores

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

# Post Processing


In [ ]:
label2class = {'0': 'person', '1': 'bicycle', '2': 'car', '3': 'motorcycle', '4': 'airplane', '5': 'bus', 
               '6': 'train', '7': 'truck', '8': 'boat', '9': 'traffic', '10': 'fire', '11': 'stop', '12': 'parking', 
               '13': 'bench', '14': 'bird', '15': 'cat', '16': 'dog', '17': 'horse', '18': 'sheep', '19': 'cow', 
               '20': 'elephant', '21': 'bear', '22': 'zebra', '23': 'giraffe', '24': 'backpack', '25': 'umbrella', 
               '26': 'handbag', '27': 'tie', '28': 'suitcase', '29': 'frisbee', '30': 'skis', '31': 'snowboard', 
               '32': 'sports', '33': 'kite', '34': 'baseball', '35': 'baseball', '36': 'skateboard', '37': 'surfboard', 
               '38': 'tennis', '39': 'bottle', '40': 'wine', '41': 'cup', '42': 'fork', '43': 'knife', '44': 'spoon', 
               '45': 'bowl', '46': 'banana', '47': 'apple', '48': 'sandwich', '49': 'orange', '50': 'broccoli', 
               '51': 'carrot', '52': 'hot', '53': 'pizza', '54': 'donut', '55': 'cake', '56': 'chair', '57': 'couch', 
               '58': 'potted', '59': 'bed', '60': 'dining', '61': 'toilet', '62': 'tv', '63': 'laptop', '64': 'mouse', 
               '65': 'remote', '66': 'keyboard', '67': 'cell', '68': 'microwave', '69': 'oven', '70': 'toaster', 
               '71': 'sink', '72': 'refrigerator', '73': 'book', '74': 'clock', '75': 'vase', '76': 'scissors', 
               '77': 'teddy', '78': 'hair', '79': 'toothbrush'}

print(list(label2class.values()))

In [ ]:
detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]

In [ ]:
%%bash
mkdir -p "detection_results_32b_CPU"
mkdir -p "detection_results_8b_DSP"
mkdir -p "prediction_results_32b_CPU"
mkdir -p "prediction_results_8b_DSP"

### Function to draw the bounding boxes

In [ ]:
from matplotlib import pyplot as plt
import cv2.dnn
import numpy as np
colors = np.random.uniform(0, 255, size=(len(list(label2class.values())), 3))
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = f'{label2class[str(class_id)]}'
    color = colors[class_id]
    img = cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    img = cv2.putText(img, label, (x +2, y -10), cv2.FONT_HERSHEY_TRIPLEX, 0.5, color, 2)
    return img

#### Actual Post Processing Code

In [ ]:
def post_proc(output, originalImagePath, save_img_path,ratio):
    #Initializing the lists
    boxes_updated = []
    scores_updated = []
    class_ids = []
    #Reading the actual image
    original_image: np.ndarray = cv2.imread(originalImagePath)

    # Preprocessing the boxes and scores
    #format of output is first 4 is the bounding boxes, 5th one is objectness score, last 80 column is score of each classes
    boxes = output[:, :4]
    scores = output[:, 4:5] * output[:, 5:]

    #Processing of bounding boxes
    boxes_xyxy = np.ones_like(boxes)
    boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
    boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
    boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
    boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
    boxes_xyxy /= ratio

    #For each prediction from 8400 predictions finding the results
    for i in range(0, output.shape[0]):
        (minScore, maxScore, minClassLoc, (x, maxClassIndex)) = cv2.minMaxLoc(scores[i])
        if maxScore >= 0.2:
            boxes_updated.append(boxes_xyxy[i])
            scores_updated.append(float(maxScore))
            class_ids.append(maxClassIndex)

    # Removing Overlapping predictions
    result_boxes = cv2.dnn.NMSBoxes(boxes_updated, scores_updated, 0.20, 0.5, 0.5) #32b CPU
    detections = []
    img = original_image

    #For each prediction showing drawing the bounding boxes
    for i in range(len(result_boxes)):
        index = result_boxes[i]
        box = boxes_updated[index]
        detection = {
            'class_id': class_ids[index],
            'class_name': label2class[str(class_ids[index])],
            'confidence': scores_updated[index],
            'box': box
             }
        detections.append(detection)
        img = draw_bounding_box(original_image, class_ids[index],detection['confidence'], int(box[0]), int(box[1]), int(box[2]), int(box[3]))                          
                                
    cv2.imwrite(save_img_path, img)


    str_pred = ""
    for dic in detections:
        str_pred+= dic['class_name']+" "+str(dic['confidence'])+" "+str(dic['box'][0])+" "+str(dic['box'][1])+" "+str(dic['box'][2])+" "+str(dic['box'][3])+"\n"
    return str_pred.strip()

In [ ]:
def postprocess_helper(outputs, img_size, p6=False):
    grids = []
    expanded_strides = []
    strides = [8, 16, 32] if not p6 else [8, 16, 32, 64]

    hsizes = [img_size[0] // stride for stride in strides]
    wsizes = [img_size[1] // stride for stride in strides]

    for hsize, wsize, stride in zip(hsizes, wsizes, strides):
        xv, yv = np.meshgrid(np.arange(wsize), np.arange(hsize))
        grid = np.stack((xv, yv), 2).reshape(1, -1, 2)
        grids.append(grid)
        shape = grid.shape[:2]
        expanded_strides.append(np.full((*shape, 1), stride))

    grids = np.concatenate(grids, 1)
    expanded_strides = np.concatenate(expanded_strides, 1)
    outputs[..., :2] = (outputs[..., :2] + grids) * expanded_strides
    outputs[..., 2:4] = np.exp(outputs[..., 2:4]) * expanded_strides

    return outputs

In [ ]:
#000000329219
folder = ["OUTPUT_32b_CPU","OUTPUT_8b_DSP"]
detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]
results = ["prediction_results_32b_CPU","prediction_results_8b_DSP"]
f=open("list.txt","r")
input_list = f.readlines()
input_shape=tuple([640,640])
f.close()
for j in range(0,len(folder)):
    for result_path in glob.glob(os.path.join(folder[j], '*')):
        if ".log" not in result_path and int(result_path.split("_")[-1])<len(input_list):
            #using raw file to traverse output folders and result folder will be in sequence as per list.txt
            raw_path = input_list[int(result_path.split("_")[-1])].replace("\n","") # rawfile path from list.txt
            if os.path.exists(raw_path):
                output = np.fromfile(result_path+'/output.raw', dtype="float32")               
                output=output.reshape((1,1,8400, 85))
                predictions = postprocess_helper(output[0], input_shape)[0]
                originalImagePath = raw_path.replace(".raw",".jpg").replace("raw","val2017")
                runtime = folder[j].split("_")[-1]
                save_img_path = detection_results[j]+"/"+raw_path.split("/")[-1].split(".")[0]+"_"+runtime+".jpg"
                ratio=dict[raw_path.split("/")[-1].split(".")[0]]
                label_and_coordinates = post_proc(predictions, originalImagePath, save_img_path,ratio)
                f = open(results[j]+"/"+raw_path.split("/")[-1].split(".")[0]+".txt","w")
                f.writelines(label_and_coordinates)
                f.close()
            else:
                print("***",raw_path)

## Visualizing the Output

In [ ]:
detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]
for CPU_path in glob.glob(os.path.join(detection_results[0], '*')):
    CPU_img_name=CPU_path.split("/")[1].split("_")[0]
    for DSP_path in glob.glob(os.path.join(detection_results[1], '*')):
        DSP_img_name=DSP_path.split("/")[1].split("_")[0]
        
        if CPU_img_name!=DSP_img_name:continue
            
        CPU_img=cv2.imread(CPU_path)
        DSP_img=cv2.imread(DSP_path)
        fing,ax=plt.subplots(1,2,figsize=(10,5))
        ax[0].imshow(CPU_img)
        ax[0].set_title("CPU 32 Inference")

        ax[1].imshow(DSP_img)
        ax[1].set_title("DSP 8 Inference")
        plt.tight_layout()
        plt.show()
